In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
from mealpy.swarm_based import GWO
from mealpy.evolutionary_based import FPA

In [2]:
import pandas as pd 

In [3]:
dataset = pd.read_csv(r"diabetes.csv")

In [4]:
X = dataset.drop(['Outcome'], axis=1)
Y = dataset['Outcome']

In [5]:
from imblearn.over_sampling import SMOTE

In [6]:
sm = SMOTE(random_state=42)
x_res, y_res = sm.fit_resample(X, Y)

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train,X_test,y_train,y_test=train_test_split(x_res, y_res,test_size=0.20,random_state=0)

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(X_train)
x_test_scaled = scaler.fit_transform(X_test)

In [10]:
X_train=x_train_scaled
X_test=x_test_scaled

In [11]:
dataset = [X_train, y_train, X_test, y_test]

In [12]:
class HybridMlp:

    def __init__(self, dataset, n_hidden_nodes, epoch, pop_size):
        self.X_train, self.y_train, self.X_test, self.y_test = dataset[0], dataset[1], dataset[2], dataset[3]
        self.n_hidden_nodes = n_hidden_nodes
        self.epoch = epoch
        self.pop_size = pop_size

        self.n_inputs = self.X_train.shape[1]
        self.model, self.problem_size, self.n_dims, self.problem = None, None, None, None
        self.optimizer, self.solution, self.best_fit = None, None, None

    def create_network(self):
        # create model
        model = Sequential()
        model.add(Dense(self.n_hidden_nodes[0], input_dim=self.n_inputs, activation='relu'))
        model.add(Dense(self.n_hidden_nodes[1], activation='relu'))
        model.add(Dense(1, activation='sigmoid'))
        # Compile model
        # model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        self.model = model
        self.problem_size = self.n_dims = np.sum([np.size(w) for w in self.model.get_weights()])

    def create_problem(self):
        self.problem = {
            "fit_func": self.fitness_function,
            "lb": [-1, ] * self.n_dims,
            "ub": [1, ] * self.n_dims,
            "minmax": "max",
            "log_to": None,
            "save_population": False
        }

    def decode_solution(self, solution):
        # solution: is a vector.
        # solution = [w11, w21, w31, w12, w22, w32, b1, b2, wh11, wh21, wh12, wh22, wh13, wh23, bo1, bo2, bo3 ]
        # number of weights = n_inputs * n_hidden_nodes + n_hidden_nodes + n_hidden_nodes * n_outputs + n_outputs
        # we decode the solution into the neural network weights
        # we return the model with the new weight (weight from solution)
        weight_sizes = [(w.shape, np.size(w)) for w in self.model.get_weights()]
        # ( (3, 5),  15 )
        weights = []
        cut_point = 0
        for ws in weight_sizes:
            temp = np.reshape(solution[cut_point: cut_point + ws[1]], ws[0])
            # [0: 15], (3, 5),
            weights.append(temp)
            cut_point += ws[1]
        self.model.set_weights(weights)

    def prediction(self, solution, x_data):
        self.decode_solution(solution)
        return self.model.predict(x_data)

    def training(self):
        self.create_network()
        self.create_problem()
        self.optimizer = GWO.OriginalGWO(self.epoch, self.pop_size)
        # self.optimizer = FPA.OriginalFPA(self.problem, self.epoch, self.pop_size)
        self.solution, self.best_fit = self.optimizer.solve(self.problem)

    def fitness_function(self, solution):  # Used in training process
        # Assumption that we have 3 layer , 1 input layer, 1 hidden layer and 1 output layer
        # number of nodes are 3, 2, 3
        # solution = [w11, w21, w31, w12, w22, w32, b1, b2, wh11, wh21, wh12, wh22, wh13, wh23, bo1, bo2, bo3 ]
        self.decode_solution(solution)
        yhat = self.model.predict(self.X_train)
        yhat = np.argmax(yhat, axis=-1).astype('int')
        acc = accuracy_score(self.y_train, yhat)
        return acc




n_hidden_nodes = [10, 5]
epoch = 50
pop_size = 50

# define model
model = HybridMlp(dataset, n_hidden_nodes, epoch, pop_size)
# fit model
model.training()

# access to the best model - best set of weights - the final weights of neural network
# model.solution
# evaluate on test set
yhat = model.prediction(solution=model.solution, x_data=X_test)
yhat = np.argmax(yhat, axis=-1).astype('int')
acc = accuracy_score(y_test, yhat)
print('Accuracy: %.3f' % acc)


7/7 [==============================] - 0s 0s/step
Accuracy: 0.525


In [13]:
X_train_std=x_train_scaled
X_test_std=x_test_scaled

In [14]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn import datasets, metrics
from mealpy.bio_based.BBO import BaseBBO


In [15]:
# LABEL ENCODER
KERNEL_ENCODER = LabelEncoder()
KERNEL_ENCODER.fit(['linear', 'poly', 'rbf', 'sigmoid'])
# print(KERNEL_ENCODER.inverse_transform( [1, 3]))


def fitness_function(solution):
    # if kernel belongs to 0 - 0.99 ==> 0       ==> linear
    #                       2 - 2.99 ==> 2
    #                       3 - 3.99 ==> 3      ==> sigmoid

    kernel_encoded = int(solution[0])
    c = solution[1]
    kernel_decoded = KERNEL_ENCODER.inverse_transform([kernel_encoded])[0]

    svc = SVC(C=c, random_state=1, kernel=kernel_decoded)
    # Fit the model
    svc.fit(X_train_std, y_train)
    # Make the predictions
    y_predict = svc.predict(X_test_std)
    # Measure the performance
    return metrics.accuracy_score(y_test, y_predict)

problem = {
    "fit_func": fitness_function,
    "lb": [0, 0.1],
    "ub": [3.99, 1000],
    "minmax": "max",
}
p_m = 0.01
elites = 2
epoch=50
pop_size=50
model = BaseBBO(epoch,pop_size,p_m,elites)
model.solve(problem)
print(f"Best solution: {model.solution[0]}")
print(f"Best kernel: {KERNEL_ENCODER.inverse_transform([int(model.solution[0][0])])[0]}, Best c: {model.solution[0][1]}")
print(f"Best accuracy: {model.solution[1]}")
model.history.save_global_objectives_chart(filename="hello2/goc")
model.history.save_local_objectives_chart(filename="hello2/loc")
model.history.save_global_best_fitness_chart(filename="hello2/gbfc")
model.history.save_local_best_fitness_chart(filename="hello2/lbfc")
model.history.save_runtime_chart(filename="hello2/rtc")
model.history.save_exploration_exploitation_chart(filename="hello2/eec")
model.history.save_diversity_chart(filename="hello2/dc")
model.history.save_trajectory_chart(list_agent_idx=[3, 5], selected_dimensions=[3], filename="hello2/tc")

2023/09/21 04:45:47 PM, INFO, mealpy.bio_based.BBO.BaseBBO: Solving single objective optimization problem.
